# Programacion Concurrente TP2 Parte2 MPI 

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py** [1]. Para más información puede consultar la última revisión en [2].

Este ejercicio fue presentado por alumnos del curso 2020.

---
## 2.1. Ejercicio - Hola Mundo con MPI


### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para $4$ o más instancias y responda:

a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?

b) ¿Cuántas de esas instancias ejecuta la función `main()`, `initWork()` y `doWork()`?

c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?

d) ¿Cómo implementaría la función BLAS `axpy()` con este patrón?¿Sería eficiente? *Tips*: Pide solo el planteo, no la implementación.

e) ¿Qué sucede cuando solo ejecuta con una sola instancia?

f) *Punto opcional*: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?

###2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [ ]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 21.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185314 sha256=9eab15f58d6a80b68a7730ef62568aed0a6beaf2e7061ea85433f682b9b21ce4
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py


### 2.1.3. Código del programa en Lenguaje Python 

In [ ]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   5#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   0#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()  # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran 
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000) 
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status() 
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG) 
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()


Writing Ejercicio2.py


### 2.1.4 Ejecución del programa

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   6#@param {type: "number"}
# --------------------------------------------

! mpirun --allow-run-as-root --oversubscribe -np $NRO python Ejercicio2.py


Version MPI4py utilizada: (3, 1)

------------------------------------
Sistema de trabajo Suizo:
------------------------------------

El jefe crea 20 horas de descanso de sus empleados:
[3 0 3 4 1 0 1 0 1 4 3 4 4 2 2 1 4 4 2 1]
Jefe enviando las tareas iniciales:
Jefe envia trabajo y 3 hs de descanso al empleado 1.
Soy el empleado con id 1, toca descanzo por 3 hs.
Jefe envia trabajo y 0 hs de descanso al empleado 2.
Jefe envia trabajo y 3 hs de descanso al empleado 3.
Jefe envia trabajo y 4 hs de descanso al empleado 4.
Soy el empleado con id 2, toca descanzo por 0 hs.
Jefe envia trabajo y 1 hs de descanso al empleado 5.
------------------------------------
Soy el empleado con id 3, toca descanzo por 3 hs.
Soy el empleado con id 4, toca descanzo por 4 hs.
Jefe recibe trabajo completado 0 del empleado 2.
Soy el empleado con id 5, toca descanzo por 1 hs.
Jefe envia nuevo trabajo y 0 hs de descanso al empleado 2.
Soy el empleado con id 2, toca descanzo por 0 hs.
Jefe recibe trabajo comp

---
# Bibliografía

[1] MPI4py: https://mpi4py.readthedocs.io/en/stable/

[2] La versión oficial de MPI (Versión 4): https://www.mpi-forum.org/docs/mpi-4.0/mpi40-report.pdf

